# PIPEX

## Makefile

In [13]:
%%file ../pipex_working/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/01/30 14:27:11 by dpentlan          #+#    #+#              #
#    Updated: 2023/04/29 12:19:40 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = pipex
RED='\033[0;31m'
GREEN='\033[0;32m'
NOCOLOR='\033[0m'

# ***** Sources *****

SRCS = pipex.c \
	pipex_utils_1.c \

SRCS_DIR = srcs/

# ***** Includes *****

INCS = pipex.h

INCS_DIR = includes/

# ***** Objects *****

OBJS = $(patsubst %.c,$(OBJS_DIR)%.o,$(SRCS))

OBJS_DIR = objs/

# ***** Compile and Archiver *****

CFLAGS = -Wall -Wextra -Werror -g

CC = cc

LIBFT = -Llibft -lft

# ***** Build *****

all: $(NAME)

bonus: $(BONUS)

$(NAME): $(OBJS) $(OBJS_MAN)
	@echo -e -n ${GREEN}
	@make -C libft --silent
	@echo -e ===== MAKING PIPEX =====
	@echo -e -n ${NOCOLOR}
	@$(CC) $(CFLAGS) $(OBJS) $(LIBFT) -o $(NAME)

$(OBJS_DIR)%.o: $(SRCS_DIR)%.c
	@mkdir -p $(OBJS_DIR)
	@$(CC) -c $(CFLAGS) -I$(INCS_DIR) $< -o $@

clean:
	@make -C libft clean --silent
	@rm -f $(OBJS)
	@rm -rf $(OBJS_DIR)

fclean: clean
	@make -C libft fclean --silent
	@rm -f $(NAME)
	@rm -f gdb.txt

re: fclean all


Overwriting ../pipex_working/Makefile


## HEADERS

In [106]:
%%file ../pipex_working/includes/pipex.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.h                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:57:04 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/20 12:57:37 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef PIPEX_H
# define PIPEX_H

# include "../libft/includes/libft.h"
# include <unistd.h>
# include <sys/wait.h>
# include <error.h>

typedef struct	s_pipex {
	int 		fds[2];
	pid_t		pid1;
	pid_t		pid2;
	int			argc;
	char		**argv;
	char		**envp;
	char		*cmd_abspath;
	char		**cmd_args;
}				t_pipex;

//	pipex.c
//	int main (int argc, char **argv, char **envp);

//	pipex_utils_1.c
int		px_error(t_pipex *pipex, char *err_message);
int 	px_err_args(void);
int		px_err_malloc(t_pipex *pipex);
int		ft_infile(t_pipex *pipex);
int		ft_outfile(t_pipex *pipex);
int		ft_px_init(t_pipex *pipex, int argc, char **argv, char **envp);
void	ft_free_tab(char **table);
int		px_exec_args(t_pipex *pipex);
void	px_close_fds(t_pipex *pipex);

#endif


Overwriting ../pipex_working/includes/pipex.h


## SOURCES

### pipex.c

In [118]:
%%file ../pipex_working/srcs/pipex.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.c                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:56:25 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/29 16:03:05 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	main(int argc, char **argv, char **envp)
{
	t_pipex	*pipex;
	t_pipex	s_pipex;

	pipex = &s_pipex;
	ft_px_init(pipex, argc, argv, envp);
	if (pipe(pipex->fds))
		px_error(pipex, "pipe");
	ft_infile(pipex);

// fork loop start here //
	px_exec_args(pipex);
	pipex->pid1 = fork();
	if (pipex->pid1 < 0)
		px_error(pipex, "fork");
	else if (pipex->pid1 == 0)
	{
		dup2(pipex->fds[1], STDOUT_FILENO);
		close(pipex->fds[0]);
		close(pipex->fds[1]);
		execve(pipex->cmd_abspath, pipex->cmd_args, NULL);
	}
	waitpid(pipex->pid1, NULL, 0);
	if (pipex->cmd_args)
		ft_free_tab(pipex->cmd_args);
// fork loop end here //

	ft_outfile(pipex);

	px_exec_args(pipex);
	pipex->pid2 = fork();
	if (pipex->pid2 < 0)
		px_error(pipex, "fork");
	if (pipex->pid2 == 0)
	{
		dup2(pipex->fds[0], STDIN_FILENO);
		close(pipex->fds[0]);
		close(pipex->fds[1]);
		execve(pipex->cmd_abspath, pipex->cmd_args, NULL);
	}
	if (pipex->cmd_args)
		ft_free_tab(pipex->cmd_args);

	px_close_fds(pipex);
	waitpid(pipex->pid2, NULL, 0);
	return (0);
}


Overwriting ../pipex_working/srcs/pipex.c


### pipex_utils_1.c

In [114]:
%%file ../pipex_working/srcs/pipex_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_utils_1.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/29 17:20:54 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/09 15:31:54 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	px_error(t_pipex *pipex, char *err_message)
{
	if (pipex->cmd_args)
		ft_free_tab(pipex->cmd_args);
	perror(err_message);
	px_close_fds(pipex);
	exit(EXIT_FAILURE);
}

int px_err_args(void)
{
	ft_putstr_fd("Usage: pipex [infile] ['command'] ... [outfile]\n", 1);
	exit(EXIT_FAILURE);
}

int	ft_infile(t_pipex *pipex)
{
	int		in_file;

	in_file = 0;
	if (access(pipex->argv[1], F_OK | R_OK))
		px_error(pipex, "access");
	in_file = open(pipex->argv[1], O_RDONLY);
	if (in_file < 0)
		px_error(pipex, "open");
	dup2(in_file, STDIN_FILENO);
	close(in_file);
	return (0);
}

int	ft_outfile(t_pipex *pipex)
{
	int		out_file;

	out_file = 0;
	out_file = open(pipex->argv[pipex->argc - 1], O_CREAT | O_WRONLY, 0666);
	if (out_file < 0)
		px_error(pipex, "open");
	dup2(out_file, STDOUT_FILENO);
	close(out_file);
	return (0);
}

int	ft_px_init(t_pipex *pipex, int argc, char **argv, char **envp)
{
	if (argc < 5)
		return (px_err_args(), 1);
	pipex->argc = argc;
	pipex->argv = argv;
	pipex->envp = envp;
	return (0);
}

void	ft_free_tab(char **table)
{
	int	i;

	i = 0;
	while (table[i])
	{
		free(table[i]);
		i++;
	}
	free(table);
	return ;
}

char	*px_get_path(char **envp)
{
	char	*path;

	path = 0;
	while (*envp)
	{
		path = ft_strnstr(*envp, "PATH=", 5);
		if (path)
			return (path);
		envp++;
	}
	return (0);
}

void	px_get_execargs(t_pipex *pipex, int com_num)
{
	pipex->cmd_args = ft_split(pipex->argv[com_num], ' ');
	if (!pipex->cmd_args)
		px_error(pipex, "malloc");
}

void	px_get_abspath(t_pipex *pipex)
{
	if (pipex->cmd_args[0][0] == '/')
		pipex->cmd_abspath = pipex->cmd_args[0];
	else
	{

	}
	if (access(pipex->cmd_abspath, F_OK | X_OK))	
		px_error(pipex, "access");
}

int	px_exec_args(t_pipex *pipex)
{
	static int	com_num;

	if (pipex) {}
	if (com_num == 0)
		com_num = 2;
	else
		com_num++;
	px_get_execargs(pipex, com_num);
	//ft_printf("pipex->cmd_args[0] = %s\n", pipex->cmd_args[0]);
	px_get_abspath(pipex);
	return (0);
}

void	px_close_fds(t_pipex *pipex)
{
	close(STDIN_FILENO);
	close(STDOUT_FILENO);
	close(STDERR_FILENO);
	close(pipex->fds[0]);
	close(pipex->fds[1]);
}


Overwriting ../pipex_working/srcs/pipex_utils_1.c


### ps_utils_notused

In [2]:
char	*px_add_abs_path(char *command, char **envp)
{
	char	**path_tab;
	char	**path_tab_start;
	char	*path;
	char	*full_path;
	char	*fuller_path;

	full_path = 0;
	fuller_path = 0;
	path = 0;
	path_tab = 0;
	path = px_get_path(envp);
	if (!path)
		return (0);
	path = path + 5;
	path_tab = ft_split(path, ':');
	if (!path_tab)
		return (0);
	path_tab_start = path_tab;
	while (*path_tab)
	{
		full_path = ft_strjoin(*path_tab, "/");
		if (!full_path)
		{
			ft_free_tab(path_tab_start);
			return (0);
		}
		fuller_path = ft_strjoin(full_path, command);
		free(full_path);
		//ft_printf("fuller_path = %s\n", fuller_path);
		if (!fuller_path)
		{
			ft_free_tab(path_tab_start);
			return (0);
		}
		if (!access(fuller_path, F_OK | X_OK))
		{
			ft_free_tab(path_tab_start);
			return (fuller_path);
		}
		free(fuller_path);
		path_tab++;
	}
	ft_free_tab(path_tab_start);
	//ft_printf("bruhhhh.\n");
	return (0);
}

char	*px_env_check(int com, char **argv, char **envp)
{
	char	**buf_table;
	char	*full_path;

	full_path = 0;
	buf_table = ft_split(argv[com], ' ');
	if (!buf_table || !buf_table[0])
		return (0);
	//ft_printf("%s\n", buf_table[0]);
	if (buf_table[0][0] != '/')
		full_path = px_add_abs_path(buf_table[0], envp);
	ft_free_tab(buf_table);
	return (full_path);
}

int	px_access_check(int argc, char **argv, char **envp)
{
	int		i;
	char	*full_path;

	i = 2;
	if (access(argv[1], F_OK | W_OK))
		return (-1);
	if (access(argv[argc - 1], F_OK | W_OK))
		return (-1);
	while (i < argc - 1)
	{
		full_path = px_env_check(i, argv, envp);
		if (!full_path)
		{
			free(full_path);
			return (-1);
		}
		free(full_path);
		i++;
	}
	return (0);
}

SyntaxError: invalid syntax (2811157015.py, line 3)

## TESTING

### Test Build

In [2]:
%%bash
cd ../pipex_working
make re

===== MAKING LIBFT =====
===== MAKING PIPEX =====


In [34]:
%%bash
cd ../pipex_working
make re > /dev/null
./pipex > /dev/null
nm -Duj pipex | grep -v "^_"
make fclean

access@GLIBC_2.2.5
close@GLIBC_2.2.5
dup2@GLIBC_2.2.5
execve@GLIBC_2.2.5
fork@GLIBC_2.2.5
free@GLIBC_2.2.5
malloc@GLIBC_2.2.5
pipe@GLIBC_2.2.5
waitpid@GLIBC_2.2.5
write@GLIBC_2.2.5


In [2]:
%%bash
# use valgrind with --track-fds=all to see if fds are closing appropriately
valgrind --track-fds=all < file1 cat | wc > file2

==86224== Memcheck, a memory error detector
==86224== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==86224== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==86224== Command: cat
==86224== 
==86224== 
==86224== FILE DESCRIPTORS: 0 open (0 std) at exit.
==86224== 
==86224== HEAP SUMMARY:
==86224==     in use at exit: 0 bytes in 0 blocks
==86224==   total heap usage: 1,051 allocs, 1,051 frees, 205,320 bytes allocated
==86224== 
==86224== All heap blocks were freed -- no leaks are possible
==86224== 
==86224== For lists of detected and suppressed errors, rerun with: -s
==86224== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)
